# 암환자 유전체 데이터 기반 암종 분류 AI 모델 개발


- '2024 생명연구자원 AI활용 경진대회'는 바이오 데이터를 기반으로 한 AI 기술의 문제 해결 능력을 탐구하는 것을 목표로 합니다. <br>이 대회는 바이오 분야에서 AI 활용의 저변을 확대하고, 복잡한 바이오 데이터를 효율적으로 분석 및 해석할 수 있는 AI 알고리즘 개발에 초점을 맞추고 있습니다. <br><br>
- 본 대회의 구체적인 과제는 암환자 유전체 데이터의 변이 정보를 활용하여 암종을 분류하는 AI 모델을 개발하는 것입니다. <br>참가자들은 제공된 학습 데이터셋(암환자 유전체 변이 정보)을 사용하여 특정 변이 정보를 바탕으로 암종을 정확하게 분류할 수 있는 AI 알고리즘을 개발해야 합니다. <br><br>
- 이 대회의 궁극적인 목적은 바이오 데이터의 활용도를 높이고, 바이오 분야에서 AI 기술의 적용 가능성을 극대화하며, 인공지능 기술이 실제 바이오 의료 문제 해결에 어떻게 기여할 수 있는지 탐구하는 것입니다.

# Import library

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import xgboost as xgb
import numpy as np

# Load Data 

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

### 모든 행이 WT인 컬럼 제거 (train, test 모두)

In [3]:
# 두 데이터프레임에서 모든 값이 'WT'인 컬럼을 각각 찾음
columns_all_wt_train = train.columns[(train == 'WT').all(axis=0)]
columns_all_wt_test = test.columns[(test == 'WT').all(axis=0)]

# train과 test에서 모든 값이 'WT'인 컬럼의 합집합 찾기
union_columns = columns_all_wt_train.union(columns_all_wt_test)

# 합집합에 해당하는 컬럼을 train과 test에서 삭제
train = train.drop(columns=union_columns)
test = test.drop(columns=union_columns)

# 결과 확인
# print("모든 값이 'WT'인 컬럼을 삭제한 train 데이터프레임:")
# print(train.head())
# print("모든 값이 'WT'인 컬럼을 삭제한 test 데이터프레임:")
# print(test.head())


### 표기 규칙 수정 (47s)

In [4]:
import pandas as pd
import re

# 돌연변이 표기 수정 규칙 적용 함수
def process_mutations(mutation_string):
    if pd.isnull(mutation_string):
        return None  # 결측치 처리

    # 여러 개의 변이를 ',' 또는 ' ' 로 나누어 처리
    mutations = mutation_string.split(',')
    processed_mutations = []

    for mutation in mutations:
        mutation = mutation.strip()
        
        # 다중 변이 패턴: '711_712FL>FL' -> 'F711F L712L'로 수정
        multi_match = re.match(r'^(\d+_\d+)([A-Z]+)>([A-Z*]+)$', mutation)
        if multi_match:
            positions = multi_match.group(1)  # '197_198'
            from_aa_seq = multi_match.group(2)  # 'YQ'
            to_aa_seq = multi_match.group(3)    # '**' 또는 특정 아미노산 코드

            pos_list = positions.split('_')
            new_mutation = []
            for i, (from_aa, to_aa) in enumerate(zip(from_aa_seq, to_aa_seq)):
                # '*'와 같은 특수한 문자가 들어오면 그대로 유지
                if to_aa == '*':
                    new_mutation.append(f"{from_aa}{pos_list[i]}*")
                else:
                    new_mutation.append(f"{from_aa}{pos_list[i]}{to_aa}")
            processed_mutations.append(' '.join(new_mutation))

        # '*' 돌연변이 패턴은 그대로 추가
        elif '*' in mutation:
            processed_mutations.append(mutation)
        
        # 기본적인 패턴: 그대로 추가
        else:
            processed_mutations.append(mutation)

    # 중복 제거 후 반환
    unique_mutations = list(dict.fromkeys(processed_mutations))  # 중복 제거
    return ', '.join(unique_mutations)

In [5]:
train_processed = train.applymap(process_mutations)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_18456\1431569657.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_processed = train.applymap(process_mutations)


In [6]:
test_processed = test.applymap(process_mutations)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_18456\4212444730.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_processed = test.applymap(process_mutations)


### misense driver 적용 (1m 26s)

In [7]:
misense_driver = pd.read_csv('missense_driver.csv', sep='\t')

In [8]:
misense_driver['Gene name'] = misense_driver['Gene name'].str.split('_').str[0]

In [9]:
yes = train.drop(columns=['ID','SUBCLASS']).columns

In [10]:
misense_driver = misense_driver[misense_driver['Gene name'].isin(yes)]

In [11]:
misense_driver_dict = misense_driver.groupby('Gene name')['Mutation'].apply(lambda x: np.array(x)).to_dict()

In [12]:
import pandas as pd

# 다중 돌연변이를 처리하는 함수
def update_misense_driver(train, misense_driver_dict):
    # 유전자 열만 추출 ('ID', 'SUBCLASS' 제외)
    gene_columns = [col for col in train.columns if col not in ['ID', 'SUBCLASS']]
    
    # 각 유전자 열에 대해 처리
    for gene in gene_columns:
        if gene in misense_driver_dict:  # misense_driver_dict에 해당 유전자가 존재하는지 확인
            # 해당 유전자의 돌연변이 리스트 가져오기
            mutation_list = misense_driver_dict[gene]
            
            # 해당 열에 대해 처리
            def process_mutations(mutation_string):
                if pd.isnull(mutation_string):  # 결측치 처리
                    return mutation_string

                # 돌연변이 문자열을 공백으로 분리하여 각 변이를 개별적으로 처리
                mutations = mutation_string.split()
                
                # 각 변이가 mutation_list에 있는지 확인하여 30으로 교체
                processed_mutations = [str(30) if mut in mutation_list else mut for mut in mutations]
                
                # 변이를 다시 공백으로 연결하여 반환
                return ' '.join(processed_mutations)
            
            # apply 함수를 사용하여 각 셀에 대해 처리
            train[gene] = train[gene].apply(process_mutations)
    
    return train


In [13]:
# 함수 적용
train_processed2 = update_misense_driver(train_processed, misense_driver_dict)
test_processed2 = update_misense_driver(test_processed, misense_driver_dict)



In [14]:
train_processed2[train_processed2['A2M'] == '30']

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
1758,TRAIN_1758,STES,30,WT,WT,WT,WT,WT,WT,WT,...,WT,T128T,WT,WT,M456T,30,WT,WT,WT,WT
1802,TRAIN_1802,LUAD,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1818,TRAIN_1818,COAD,30,WT,WT,WT,WT,WT,WT,WT,...,A1913D,WT,WT,WT,WT,WT,WT,WT,WT,WT
2477,TRAIN_2477,BRCA,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,R162C,WT,WT,WT,WT
3729,TRAIN_3729,COAD,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [15]:
test_processed2[test_processed2['A2M'] == '30']

,ID,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
2370,TEST_2370,30,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,V1179I,WT,WT,WT,WT,WT


### missense passenger 적용 (2m 44s)

In [16]:
misense_passenger = pd.read_csv('missense_passenger.csv', sep='\t')

In [17]:
misense_passenger['Gene name'] = misense_passenger['Gene name'].str.split('_').str[0]

In [18]:
yes = train.drop(columns=['ID','SUBCLASS']).columns

In [19]:
misense_passenger = misense_passenger[misense_passenger['Gene name'].isin(yes)]

In [20]:
misense_passenger_dict = misense_passenger.groupby('Gene name')['Mutation'].apply(lambda x: np.array(x)).to_dict()

In [21]:
def update_misense_passenger(train, misense_passenger_dict):
    # 유전자 열만 추출 ('ID', 'SUBCLASS' 제외)
    gene_columns = [col for col in train.columns if col not in ['ID', 'SUBCLASS']]
    
    # 각 유전자 열에 대해 처리
    for gene in gene_columns:
        if gene in misense_passenger_dict:  # misense_driver_dict에 해당 유전자가 존재하는지 확인
            # 해당 유전자의 돌연변이 리스트 가져오기
            mutation_list = misense_passenger_dict[gene]
            
            # 해당 열에 대해 값이 돌연변이 리스트에 있는지 확인하고 있으면 30으로 대체
            #train[gene] = train[gene].apply(lambda x: 0 if x in mutation_list else x )
            train[gene] = train[gene].apply(lambda x: str(0) if not isinstance(x, (int, float)) and x in mutation_list else x)

    
    return train


In [22]:
train_processed3=update_misense_passenger(train_processed2,misense_passenger_dict)
test_processed3=update_misense_passenger(test_processed2,misense_passenger_dict)

In [23]:
train_processed3[train_processed3['A2M'] == '0']

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
120,TRAIN_0120,KIPAN,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
318,TRAIN_0318,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
669,TRAIN_0669,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2136,TRAIN_2136,LUAD,0,WT,WT,WT,0,WT,WT,WT,...,WT,0,WT,WT,WT,WT,WT,WT,WT,WT
2306,TRAIN_2306,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2389,TRAIN_2389,COAD,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2510,TRAIN_2510,LUAD,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,E291E,WT,WT,WT
2746,TRAIN_2746,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2969,TRAIN_2969,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2984,TRAIN_2984,COAD,0,WT,WT,0,S2192N F1854F K376R,WT,WT,V1698V E1295* S1071L,...,0,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [24]:
test_processed3[test_processed3['A2M'] == '0']

,ID,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
1525,TEST_1525,0,WT,WT,WT,L317fs,A2200V A2230V,R20W,WT,WT,...,WT,Q288Q,WT,WT,R992C,0,WT,WT,WT,WT
1873,TEST_1873,0,WT,WT,WT,WT,WT,0,0,WT,...,WT,WT,0,WT,WT,0,WT,WT,WT,WT


### silent driver 적용 (51s)

In [25]:
silent_driver = pd.read_csv('silent_driver.csv', sep='\t')

In [26]:
silent_driver['Gene name'] = silent_driver['Gene name'].str.split('_').str[0]

In [27]:
yes = train.drop(columns=['ID','SUBCLASS']).columns

In [28]:
silent_driver = silent_driver[silent_driver['Gene name'].isin(yes)]

In [29]:
silent_driver_dict = silent_driver.groupby('Gene name')['Mutation'].apply(lambda x: np.array(x)).to_dict()

In [30]:
def update_silent_driver(train, silent_driver_dict):
    # 유전자 열만 추출 ('ID', 'SUBCLASS' 제외)
    gene_columns = [col for col in train.columns if col not in ['ID', 'SUBCLASS']]
    
    # 각 유전자 열에 대해 처리
    for gene in gene_columns:
        if gene in silent_driver_dict:  # misense_driver_dict에 해당 유전자가 존재하는지 확인
            # 해당 유전자의 돌연변이 리스트 가져오기
            mutation_list = silent_driver_dict[gene]
            
            # 해당 열에 대해 값이 돌연변이 리스트에 있는지 확인하고 있으면 30으로 대체
            #train[gene] = train[gene].apply(lambda x: 0 if x in mutation_list else x )
            train[gene] = train[gene].apply(lambda x: str(30) if not isinstance(x, (int, float)) and x in mutation_list else x)

    
    return train


In [31]:
train_processed4 = update_silent_driver(train_processed3,silent_driver_dict)
test_processed4 = update_silent_driver(test_processed3,silent_driver_dict)

In [32]:
train_processed4[train_processed4['A2M'] == '30']

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
594,TRAIN_0594,CESC,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1758,TRAIN_1758,STES,30,WT,WT,WT,WT,WT,WT,WT,...,WT,T128T,WT,WT,M456T,30,WT,WT,WT,WT
1802,TRAIN_1802,LUAD,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1818,TRAIN_1818,COAD,30,WT,WT,WT,WT,WT,WT,WT,...,0,WT,WT,WT,WT,WT,WT,WT,WT,WT
2477,TRAIN_2477,BRCA,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,R162C,WT,WT,WT,WT
3729,TRAIN_3729,COAD,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [33]:
test_processed4[test_processed4['A2M'] == '30']

,ID,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
2370,TEST_2370,30,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,V1179I,WT,WT,WT,WT,WT


### silent passenger 적용 (2m 42s)

In [34]:
silent_passenger = pd.read_csv('silent_passenger.csv', sep='\t')

In [35]:
silent_passenger['Gene name'] = silent_passenger['Gene name'].str.split('_').str[0]

In [36]:
yes = train.drop(columns=['ID','SUBCLASS']).columns

In [37]:
silent_passenger = silent_passenger[silent_passenger['Gene name'].isin(yes)]

In [38]:
silent_passenger_dict = silent_passenger.groupby('Gene name')['Mutation'].apply(lambda x: np.array(x)).to_dict()

In [39]:
def update_silent_passenger(train, silent_passenger_dict):
    # 유전자 열만 추출 ('ID', 'SUBCLASS' 제외)
    gene_columns = [col for col in train.columns if col not in ['ID', 'SUBCLASS']]
    
    # 각 유전자 열에 대해 처리
    for gene in gene_columns:
        if gene in silent_passenger_dict:  # misense_driver_dict에 해당 유전자가 존재하는지 확인
            # 해당 유전자의 돌연변이 리스트 가져오기
            mutation_list = silent_passenger_dict[gene]
            
            # 해당 열에 대해 값이 돌연변이 리스트에 있는지 확인하고 있으면 30으로 대체
            #train[gene] = train[gene].apply(lambda x: 0 if x in mutation_list else x )
            train[gene] = train[gene].apply(lambda x: str(0) if not isinstance(x, (int, float)) and x in mutation_list else x)

    
    return train


In [40]:
train_processed5 = update_silent_passenger(train_processed4,silent_passenger_dict)
test_processed5 = update_silent_passenger(test_processed4,silent_passenger_dict)

In [41]:
train_processed5[train_processed5['A2M'] == '0']

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
120,TRAIN_0120,KIPAN,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
318,TRAIN_0318,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
669,TRAIN_0669,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1502,TRAIN_1502,COAD,0,WT,WT,0,WT,WT,WT,V1774V T311T,...,K162T S664S A779V T1526M L1806V R2115*,WT,WT,WT,Q1572Q H730Q I688I,WT,0,WT,WT,WT
1606,TRAIN_1606,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2136,TRAIN_2136,LUAD,0,WT,WT,WT,0,WT,WT,WT,...,WT,0,WT,WT,WT,WT,WT,WT,WT,WT
2306,TRAIN_2306,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2389,TRAIN_2389,COAD,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2510,TRAIN_2510,LUAD,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,0,WT,WT,WT
2746,TRAIN_2746,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [42]:
test_processed5[test_processed5['A2M'] == '0']

,ID,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
1337,TEST_1337,0,WT,WT,WT,WT,WT,WT,WT,WT,...,K1345*,WT,WT,WT,WT,WT,WT,WT,WT,A430E
1525,TEST_1525,0,WT,WT,WT,L317fs,A2200V A2230V,R20W,WT,WT,...,WT,Q288Q,WT,WT,R992C,0,WT,WT,WT,WT
1873,TEST_1873,0,WT,WT,WT,WT,WT,0,0,WT,...,WT,WT,0,WT,WT,0,WT,WT,WT,WT
2214,TEST_2214,0,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


### amino acid 함수 정의 + train,test set 적용 (1m 26s)

In [43]:
#aminoacid version3
# 변이 유형을 분류하는 함수 정의
import pandas as pd
import re
amino_acid_properties = {
    'A': 'nonpolar',    # Alanine
    'R': 'positive',    # Arginine
    'N': 'polar',       # Asparagine
    'D': 'negative',    # Aspartic Acid
    'C': 'polar',       # Cysteine
    'Q': 'polar',       # Glutamine
    'E': 'negative',    # Glutamic Acid
    'G': 'nonpolar',    # Glycine
    'H': 'positive',    # Histidine
    'I': 'nonpolar',    # Isoleucine
    'L': 'nonpolar',    # Leucine
    'K': 'positive',    # Lysine
    'M': 'nonpolar',    # Methionine
    'F': 'aromatic',    # Phenylalanine
    'P': 'nonpolar',    # Proline
    'S': 'polar',       # Serine
    'T': 'polar',       # Threonine
    'W': 'aromatic',    # Tryptophan
    'Y': 'aromatic',    # Tyrosine
    'V': 'nonpolar',    # Valine
}

def classify_mutation(mutation):
    # 결측치 처리
    if pd.isnull(mutation):
        return None  # 또는 특정 코드로 지정 가능

    # 숫자인 경우 또는 숫자로 변환 가능한 경우 그대로 반환
    try:
        mutation = float(mutation)  # 숫자로 변환 시도
        return int(mutation) if mutation.is_integer() else mutation  # 정수형이면 int, 실수형이면 그대로 반환
    except ValueError:
        pass  # 숫자가 아닌 경우 문자열 처리로 넘어감

    mutation = str(mutation).strip()

    # WT 체크
    if mutation == 'WT':
        return 0  # WT (Wild Type)

    # 프레임시프트 돌연변이 체크 ('fs' 포함)
    if 'fs' in mutation:
        return 30  # 프레임시프트 돌연변이

    # 중단 돌연변이 체크 ('*' 포함)
    if '*' in mutation:
        return 30  # 중단 돌연변이

    # 삭제 돌연변이 체크 ('del' 포함)
    if 'del' in mutation:
        return 30  # 삭제 돌연변이 (Deletion mutation)

    # 단일 아미노산 변이 패턴 매칭 (예: 'R496Q', 'L1700L')
    match = re.match(r'^([A-Z])(\d*)([A-Z])$', mutation)
    if match:
        from_aa = match.group(1)  # 원래 아미노산
        position = match.group(2) # 위치 (사용하지 않음)
        to_aa = match.group(3)    # 변이된 아미노산

        # 침묵 돌연변이 체크 (아미노산이 동일한 경우)
        if from_aa == to_aa:
            return 0 # 침묵 돌연변이

        # 아미노산 성질 가져오기
        from_property = amino_acid_properties.get(from_aa)
        to_property = amino_acid_properties.get(to_aa)

        # 아미노산 코드가 유효한지 확인
        if from_property is None or to_property is None:
            return 7  # 알 수 없는 아미노산 코드

        # 보존적 돌연변이 체크 (아미노산 성질이 동일한 경우)
        if from_property == to_property:
            return 2  # 보존적 돌연변이
        else:
            return 3  # 비보존적 돌연변이
    else:
        # 패턴 매칭 실패한 경우
        return 7  # 매칭 실패한 경우 7 반환

# 다중 치환을 처리하는 함수 정의
def classify_multiple_mutations(mutation_string):
    # 결측치 처리
    if pd.isnull(mutation_string):
        return None  # 또는 특정 코드로 지정 가능

    # 변이 문자열을 공백으로 분리
    mutations = set(str(mutation_string).strip().split())

    labels = []
    for mutation in mutations:
        # 숫자인 경우 또는 숫자로 변환 가능한 경우 그대로 유지
        try:
            mutation = float(mutation)  # 숫자로 변환 시도
            mutation = int(mutation) if mutation.is_integer() else mutation  # 정수형이면 int, 실수형이면 그대로 유지
            labels.append(mutation)
        except ValueError:
            # 숫자가 아닌 경우 기존 로직 사용
            label = classify_mutation(mutation)
            if label is not None:
                labels.append(label)

    if labels:
        # 합계 반환
        return sum(labels)
    else:
        return None  # 또는 특정 코드로 지정 가능


In [44]:
# 제외할 열 목록 (예시로 'ID'와 'SUBCLASS'를 제외)
exclude_cols = ['ID', 'SUBCLASS']

# 변이 데이터가 있는 열 목록
mutation_cols = [col for col in train_processed5.columns if col not in exclude_cols]

# 각 열에 함수 적용
for col in mutation_cols:
    train_processed5[col] = train_processed5[col].apply(classify_multiple_mutations)


In [45]:
# 제외할 열 목록 (예시로 'ID'와 'SUBCLASS'를 제외)
exclude_cols = ['ID', 'SUBCLASS']

# 변이 데이터가 있는 열 목록
mutation_cols = [col for col in test_processed5.columns if col not in exclude_cols]

# 각 열에 함수 적용
for col in mutation_cols:
    test_processed5[col] = test_processed5[col].apply(classify_multiple_mutations)


### CSV 저장 & 불러오기
- 함수 적용 시간이 오래걸리기때문에 여기까지 trian을 csv로 저장 후 불러내기 해서 사용
- 저장한 train파일에 컬럼(유전자) 수는 원래 유전자수(4865)에서 모든 행이 'WT'이 제거됨 (4226열)
- 저장된 파일은 전처리가 끝난 상태 -> 모델만 수정해가면서 train,test 불러와서 반복해서 돌리기


In [46]:
#4개의 csv파일함수적용 후 'PS_mapping_ver3.csv'로 저장하기 ( 최초 1회만 실행)
train_processed5.to_csv('PS_mapping_ver3.csv')
test_processed5.to_csv('test_wtcut.csv')

In [1]:
# 두번째부터 여기부터 실행하기 train,test 는 전처리가 끝났음, 바로 모델에 학습시키면 됨
import pandas as pd
train = pd.read_csv('./open/train_preprocessed.csv')
train = train.iloc[:,1:]
test = pd.read_csv('./open/test_preprocessed.csv')
test = test.iloc[:,1:]

# Data Preprocessing

In [2]:
from sklearn.preprocessing import LabelEncoder
le_subclass = LabelEncoder()
train['SUBCLASS'] = le_subclass.fit_transform(train['SUBCLASS'])

# Model definition

- aminoacid version 3 ( 공통 'WT' 제거 했을 때)
- Train Accuracy: 74.64%
- Test Accuracy: 36.66%

In [3]:
train.head()

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,TRAIN_0001,19,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,TRAIN_0002,20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,TRAIN_0003,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TRAIN_0004,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
test.head()

,ID,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TEST_0000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,TEST_0001,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,30,0,0,0,0
2,TEST_0002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,TEST_0003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TEST_0004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# 1. 데이터 준비
X = train.drop(columns=['ID','SUBCLASS'])
y = train['SUBCLASS']

In [6]:
# 65*65 이미지로 변환

import numpy as np
# 데이터프레임의 행 개수 구하기
num_rows = X.shape[0]

# 각 행을 3차원 배열로 분리
array_3d = X.values.reshape((num_rows, 1, -1))

# (6201, 65, 65) 배열로 변환
# reshaped_array = array_3d.reshape(num_rows, 65, 65)
# (6201, 65, 65,1) 배열로 변환
reshaped_array = array_3d.reshape(num_rows, 65, 65,1)

print("변환된 배열의 형태:", reshaped_array.shape)
print(reshaped_array)

변환된 배열의 형태: (6201, 65, 65, 1)
[[[[ 0]
   [ 0]
   [ 0]
   ...
   [ 0]
   [ 0]
   [ 0]]

  [[ 0]
   [ 0]
   [ 0]
   ...
   [ 0]
   [ 0]
   [ 0]]

  [[ 0]
   [ 0]
   [ 0]
   ...
   [ 0]
   [ 0]
   [ 0]]

  ...

  [[ 0]
   [ 0]
   [ 0]
   ...
   [ 0]
   [ 0]
   [ 0]]

  [[ 0]
   [ 0]
   [ 0]
   ...
   [ 0]
   [ 0]
   [ 0]]

  [[ 0]
   [ 0]
   [ 0]
   ...
   [ 0]
   [ 0]
   [ 0]]]


 [[[ 0]
   [ 0]
   [ 0]
   ...
   [ 0]
   [ 0]
   [ 0]]

  [[ 0]
   [ 0]
   [ 0]
   ...
   [ 0]
   [ 0]
   [ 0]]

  [[ 0]
   [ 0]
   [ 0]
   ...
   [ 0]
   [ 0]
   [ 0]]

  ...

  [[ 0]
   [ 0]
   [ 0]
   ...
   [ 0]
   [ 0]
   [ 0]]

  [[ 0]
   [ 0]
   [ 0]
   ...
   [ 0]
   [ 0]
   [ 0]]

  [[ 0]
   [ 0]
   [ 0]
   ...
   [ 0]
   [ 0]
   [ 0]]]


 [[[ 0]
   [ 0]
   [ 0]
   ...
   [ 0]
   [ 0]
   [ 0]]

  [[ 0]
   [ 0]
   [ 0]
   ...
   [ 0]
   [ 0]
   [ 0]]

  [[ 0]
   [ 0]
   [ 0]
   ...
   [ 0]
   [ 0]
   [ 0]]

  ...

  [[ 0]
   [ 0]
   [ 0]
   ...
   [ 0]
   [ 0]
   [ 0]]

  [[ 0]
   [ 0]
   [ 0]
   ...
  

In [7]:
import pickle
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
import collections
import matplotlib.pyplot as plt
import pandas as pd
import keras
import tensorflow as tf
# from tf.keras.models import Sequential, Model
# from tf.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten, Input
# from tf.keras.callbacks import EarlyStopping, ModelCheckpoint
# from tf.keras.layers.normalization import BatchNormalization
# from tf.keras.layers.advanced_activations import LeakyReLU
# from tf.sklearn.metrics import precision_recall_curve, roc_curve, auc, average_precision_score
# from sklearn.model_selection import StratifiedKFold

2024-10-13 14:36:08.651834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-13 14:36:08.716248: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-13 14:36:08.735017: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-13 14:36:08.859242: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-13 14:36:10.097874: W tensorflow/compiler/tf2

In [8]:
x_train_val,x_test,y_train_val,y_test = train_test_split(reshaped_array,y,random_state=42,test_size=0.2,stratify=y)
x_train,x_val,y_train,y_val = train_test_split(x_train_val,y_train_val,random_state=42,test_size=0.2,stratify=y_train_val)

In [9]:
x_train.shape, y_train.shape

((3968, 65, 65, 1), (3968,))

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense, AveragePooling2D, AveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping

tf.random.set_seed(42)
batch_size = 64
epochs = 100
num_classes = len(y.unique())

# 모델 정의
model = Sequential([
    # 입력 데이터 형태는 (65, 65, 1)
    # Conv2D(32, kernel_size=5, strides=1, input_shape=(65, 65, 1)),
    # Activation('mish'),
    # Conv2D(32, kernel_size=5, strides=1),
    # Activation('mish'),
    MaxPooling2D(pool_size=2),
    # AveragePooling2D(pool_size=2),
    
    # Conv2D(64, kernel_size=3, strides=1),
    # Activation('mish'),
    # Conv2D(64, kernel_size=5, strides=1),
    # Activation('mish'),
    MaxPooling2D(pool_size=2),
    # AveragePooling2D(pool_size=2),
    
    Conv2D(128, kernel_size=5, strides=1),
    Activation('mish'),
    Conv2D(128, kernel_size=5, strides=1),
    Activation('mish'),
    # MaxPooling2D(pool_size=2),
    AveragePooling2D(pool_size=2),

    # 분류 레이어
    Flatten(),
    Dense(128, activation='mish'),
    Dense(64, activation='mish'),
    Dense(num_classes, activation='softmax'),
])

model.summary()

# 모델 컴파일
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy']) # sparse_categorical_crossentropy

# 콜백 정의
callbacks = [EarlyStopping(monitor='val_accuracy', patience=5, verbose=0)]

# 모델 학습
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=(x_val, y_val))

# 모델 평가
scores = model.evaluate(x_test, y_test, verbose=1)

# 모델 저장
model.save('conv_model001.keras')

# 모델 불러오기
loaded_model = load_model('conv_model001.keras')

# 불러온 모델 평가 (확인용)
loaded_scores = loaded_model.evaluate(x_test, y_test, verbose=0)
print(loaded_scores)

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ max_pooling2d_14 (MaxPooling2D) │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_43 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_43 (Activation)      │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_44 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_44 (Activation)      │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_7             │ ?                      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.1846 - loss: 2.9613 - val_accuracy: 0.3125 - val_loss: 2.2497
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3299 - loss: 2.1168 - val_accuracy: 0.3417 - val_loss: 2.1226
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4408 - loss: 1.7729 - val_accuracy: 0.3327 - val_loss: 2.1678
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5070 - loss: 1.5197 - val_accuracy: 0.3236 - val_loss: 2.2613
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5763 - loss: 1.3053 - val_accuracy: 0.3065 - val_loss: 2.4672
Epoch 6/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6229 - loss: 1.1501 - val_accuracy: 0.3165 - val_loss: 2.5433
Epoch 7/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6625 - loss: 1.0073 - val_accuracy: 0.3196 - val_loss: 2.7235
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3266 - loss: 2.6223
[2.702336072921753, 0.31748589873313

# Inference

In [58]:
# test 전처리가 끝난 상태로 저장됐기 때문에 바로 Inference 가능
test = test.drop(columns=['ID'])

In [59]:
missing_values = test.isna().sum()
columns_with_missing_values = missing_values[missing_values > 0]
print(columns_with_missing_values)

AK2         29
ATP6V1H      1
CCRL2        2
CFP          1
CNOT2       77
CRAT         1
DPYSL4       1
GUK1         1
IER3         1
INHBB        1
KCNH1        1
MYL1        21
NDUFV1       1
NUDT4        1
POLD2        1
PTCH1        1
PTGES3       3
RBM5        19
SCAMP1       1
SCNN1A       1
SLC25A28     1
SYBU         1
TMEM97       4
TNFAIP6     65
dtype: int64


In [61]:
test_filled = test.fillna(0)
# scaler = MinMaxScaler()
# test_filled_scaled = scaler.fit_transform(test_filled)
# 데이터프레임의 행 개수 구하기
num_test_rows = test_filled.shape[0]

# 각 행을 3차원 배열로 분리
array_test_3d = test_filled.values.reshape((num_test_rows, 1, -1))

# (6201, 65, 65) 배열로 변환
reshaped_test_array = array_test_3d.reshape(num_test_rows, 65, 65)

reshaped_array = array_test_3d.reshape(num_test_rows, 65, 65,1)

print("변환된 배열의 형태:", reshaped_test_array.shape)
print(reshaped_test_array)

변환된 배열의 형태: (2546, 65, 65)
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 3. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 4. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 4. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 6. 7.]
  [3. 0. 9. ... 0. 2. 3.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 6. 2. 6.]
  [0. 0. 0. ... 2. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0.

In [62]:
predictions = model.predict(reshaped_test_array)
# (num_test_rows, 1, 4225) 배열로 변환
# reshaped_array = array_3d.reshape(num_test_rows, 1)
# print("변환된 배열의 형태:", reshaped_array.shape)

80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [63]:
original_labels = le_subclass.inverse_transform(predictions)

ValueError: y should be a 1d array, got an array of shape (2546, 26) instead.

# Submisson

In [12]:
submisson = pd.read_csv("./sample_submission.csv")

In [13]:
submisson["SUBCLASS"] = original_labels

In [14]:
submisson.to_csv('amino3_aml5.csv', encoding='UTF-8-sig', index=False)